# Project Report

## Basics of Mobile Robotics, MICRO-452

### Mehdi Krichen, Francesco Nonis, Michael Richter, Vincent Philippoz, Nathan Decurnex

# 1) Environment

Our environment in which our thymio will evolve is a rectangle space composed of two A0 sheet. The background is green to imitate grass. It has been set to a uniform dark green to not intefer with the thymio's ground sensors.
The world is also compose of roads and crossroads. The roads will be seen as obstacles. Obviously, the robot can use the crossroads to pass roads. The design of these modules stay simple: rectangular shapes. This is crucial to simplify the vision part. 


![env](images/env.jpg)

# 2) Implementation

## 2.1) Vision

The vision part of the project is essentially used at initialization. It will be used to build an initial obstacle map of the environment. The processing will also find the Thymio initial location and orientation, and the target to reach.

At the start, a picture of the whole environment is taken. The first step is to trim the initial image to obtain a frame with only the environment. We first tried to found the 4 corners of the map in order to perform a reshaping to account for perspective. But the task of finding the corners was not so simple for several reasons. We tried to have one particular shape on each corner so that we could perform template matching with it. But it was hard to detect 4 time the same shape in our environmental conditions; even by filtering our image, other shapes in our environment had regularly more correlation with the template than the real one. We thus thought to change the shape for each corner, but we were limited in the choice of easy distinguishable colors and shape. Moreover, choosing a particular shape could make hard the template detection due to perspective. It why we wanted to have circles. A last factor that made hard edge detection via templatematching is that we do not know the exact size of the template on the image; template matching is very sensitive to the size of the object we are trying to match. In other word, to make work correctly template matching, we should know in advance the size in pixels of our template on the image.  

Before trying some other fancy techniques, we wanted to have a backup. It was to simply mask the green background region of our environment via template matching. This allowed us to have a rather good cropping of the picture as long as it has been taken far away enough; to avoid having straight line deformations. The result was good enough to keep it. We could have tried line detection, but the roads or the floor pattern outside of our map would have also been detected. Sometime to stay simple is the good solution.  

Once the image trimmed, Template matching is again used to detect all the objects on the map. The 4 patterns of turning roads (4 orientations) and the two orientation of straight lines were used to detect the roads. Even with such a simple design, the sensitivity and selectivity of template matching was quite poor. Luminosity is also a decisive factor when trying to detect objects. A lot of hyperparameter needed to be tuned in order to achieve a sufficient road detection. We could have gone for simple uniform gray roads: it would have simplified the template search (also faster computation) and improved the selectivity/sensitivity. But at the beginning, a second robot was planned to follow the road: the central white line would have been used to track the path. 

The crossroads were detected by the same way as the roads. As expected, the result is also highly depending on the tuning of hyperparameters and lighting. The target position was finally decided to be a simple circle. This allowed no orientation dependence: allowing one single template search.

Finally, the Thymio robot is detected thanks to two very small circle placed strategically at the back and front of the robot. Once the circles detected by template matching, a simple calculation is done to obtain the robot hole's position and orientation. We also tried a data driven method: Convolutionnal Neural Network (CNN) and Support Vector Regression to extract the robot position from the image as input. The major issue was to collect data and label it. Simple trials with 25 data points yielded unsatisfying results and stopped our search in this direction. 


### Illustrations

In this section, we present sub-optimal results and particularities that can be observed from.

![env](images/angle1.jpg)

![env](images/angle12.jpg)

Above are illustrations on how sensitive the template matching, for a the road angle, is. The results for the four different orientation yielded the same kind of results showed above. This may be due to deformation of roads on the bottom of the image. The template matches better straight roads than certain correct road on the bottom.

This situation only happens with real big images. When we were testing on smaller scaled map or screenshots of the map on the computer, this did not happen. This shows how image processing becomes harder in real conditions. We think that the template matching could have been improved a little bit: by taking the template image with the same camera that takes the real pictures shown. Currently, the template are the computer image (numerical ones).

This problem was essentially solved by using the high sensitivity of the template matching of the straight roads. This in some way is a bit cheating, but as long as it works...

![env](images/rd2.jpg)

![env](images/road44.jpg)

We can see at which point, even the wrong orientation of the road can be detected easily. Some crossroads are also captured in the process. Even some opposite orientation roads are detected while the correct orientation is not (see second illustration on the bottom, the template orientation can be deduced: horizontal roads). 

![env](images/psg2.jpg)

Of course the same effect appears also for crossroads. The right crossroad is much more detected than the left one. If we want absolutely to detect the two horizontal crossroad, we need to decrease the threshold. This has a direct effect on the right crossroad: it appears larger; even some portion of the right crossroad combined to the road is more detectable than the left crossroad... 

The lighting is even more affecting the results than previously. We think it is because when the yellow is well lighted, it is almost seen like white from the camera. The figure shows a little bit this effect for the crossroad on the left of the image that appears brighter than the other ones.

### Real testing

If you want a real test implementation of the Image Processing part, you can run the cell below. It will print the current progression of the process and show several plots showing each step.

In [ ]:
%run IP/IP_methods.ipynb 
(x,y), orientation = pipeline_IP(plot=True)
print(f'position of thymio is ({x},{y}), the x position is on the horizontal axis and y on the vertical axis. The zero is on the bottom left')
print(f'its orientation is {orientation} [rad] (defined as [0,2Pi] with respect to the horizontal axis)') 

Globally, image processing is very hard. Robust method to lightning, object orientation, deformation or camera type etc.. do not really exist. All lot of testing needed to be done.

## 2.2) Path Planning

## 2.3) Global Navigation

## 2.4) Local Navigation

## 2.5) Localization

# 3) Conclusion 